# 如何制作自己的数据图

In [1]:
import warnings

from requests.packages import target

warnings.filterwarnings('ignore')
import torch

x是每个点的输入特征，y是每个点的标签

edge_index 是邻接矩阵的稀疏矩阵。


In [2]:
# 点                v0     v1     v2     v3
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)
# 定义边
edge_index = torch.tensor([[0, 0, 1, 2, 3],  # 起始点
                           [1, 3, 0, 1, 2]], dtype=torch.long)    # 终止点

创建pyg的图

In [3]:
from torch_geometric.data import  Data

data = Data(x=x, y=y, edge_index=edge_index)
print(data)

Data(x=[4, 2], edge_index=[2, 5], y=[4])


# 基于图神经网络的电商购买预测

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("chadgostopp/recsys-challenge-2015")

print("Path to dataset files:", path)
#df = pd.read_""

100%|██████████| 841M/841M [00:46<00:00, 19.2MB/s] 

Extracting model files...


Path to dataset files: C:\Users\Dell\.cache\kagglehub\datasets\chadgostopp\recsys-challenge-2015\versions\1


引入库
LabelEncoder：用于将分类变量（如‘item_id’）转换为数字编码。
torch 和 torch_geometric.data：用于处理和定义图数据结构。
tqdm：用于显示进度条。  

In [5]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import InMemoryDataset, Data
from tqdm import tqdm


In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

df = pd.read_csv('data/yoochoose-data/yoochoose-clicks.dat', header=None)
df.columns = ['session_id', 'timestamp', 'item_id', 'category']

buy_df = pd.read_csv('data/yoochoose-data/yoochoose-buys.dat', header=None)
buy_df.columns = ['session_id', 'timestamp', 'item_id', 'price', 'quality']

#对‘item_id’列进行编码
item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
#显示处理后的数据框前几行
df.head()


,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


场景说明:

一群人在淘宝逛，最终可能会买一些东西。

yoochoose-clicks：表示用户的浏览行为，其中一个item_id就表示一次登录都浏览了什么东西。

item_id就是他所浏览的商品，yoochoose-buys就是标签，代表他是否会购买。

session_id：表示用户。

timestamp:表示时间。

item_id:表示商品的名字。


In [7]:
import numpy as np
# 选择其中以下部分来建模
sampled_session_id = np.random.choice(df.session_id.unique(), 100000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    100000
timestamp     355670
item_id        20355
category         129
dtype: int64

比较浏览的数据df与购买的数据buy_d中重叠的部分，将他们打上标签。

In [8]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
137,43,2014-04-05T07:54:10.194Z,23573,0,False
138,43,2014-04-05T07:55:58.886Z,11505,0,False
646,212,2014-04-04T16:56:23.210Z,39932,0,False
647,212,2014-04-04T16:59:53.129Z,39932,0,False
648,212,2014-04-04T17:07:33.339Z,10902,0,False


## 制作数据集
把每一个用户session_id都当作一个graph，每一个graph具有多个node和label；

其中每个图中的node就是一个商品item_id，特征暂时用其id表示，最后会做embedding。

数据集的制作流程：
1. 首先遍历数据中的每一组session_id， 目的是将其制作成`from torch_geometric.data import Data`的格式；
2. 对每一组session_id中的所有item_id进行编码，例如：15453， 3651，15451， 就按照数值大小编码成（2，0，1）；
3. 这样编码的目的是制作edge_index，因为在edge_index中我们需要从0，1，2，3开始；
4. 点的特征就是用她的id组成，edge_index是这样，因为我们浏览过程的顺序不一定是按大小排列，例如（0，0，2，1）；
5. 所以上述对应的边就是0->0,0->2,2->1，于是对应的索引就是target_nodes:[0 2 1], source_nodes:[0 0 2];
6. 最后转换格式`data=Data(x=x, edge_index=edge_index, y=y)`
7. 最后将数据集保存下来。

In [10]:
from torch_geometric.data import InMemoryDataset
from tqdm import *
df_test = df[:100]
grouped = df_test.groupby('session_id')
for session_id, group in tqdm(grouped):
    print('session_id:', session_id)

    sess_item_id = LabelEncoder().fit_transform(group.item_id)
    print('sess_item_id:', sess_item_id)

    group = group.reset_index(drop=True)
    group['sess_item_id'] = sess_item_id
    print('group:', group)

    node_features = group.loc[group.session_id==session_id, ['sess_item_id', 'item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values
    node_features = torch.LongTensor(node_features).unsqueeze(1)
    print('node_features:', node_features)

    target_nodes = group.sess_item_id.values[1:]
    source_nodes = group.sess_item_id.values[:-1]
    print('target_nodes:', target_nodes)
    print('source_nodes:', source_nodes)

    edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    x = node_features
    y = torch.FloatTensor([group.label.values[0]])

    data = Data(x=x, edge_index=edge_index, y=y)
    print('data:', data)

100%|██████████| 26/26 [00:00<00:00, 204.62it/s]

session_id: 43
sess_item_id: [1 0]
group:    session_id                 timestamp  item_id category  label  sess_item_id
0          43  2014-04-05T07:54:10.194Z    23573        0  False             1
1          43  2014-04-05T07:55:58.886Z    11505        0  False             0
node_features: tensor([[11505],
        [23573]])
target_nodes: [0]
source_nodes: [1]
data: Data(x=[2, 1], edge_index=[2, 1], y=[1])
session_id: 212
sess_item_id: [ 6  6  1 10 10 11  5  8  0  2  9  3 12  4 12  7]
group:     session_id                 timestamp  item_id category  label  \
0          212  2014-04-04T16:56:23.210Z    39932        0  False   
1          212  2014-04-04T16:59:53.129Z    39932        0  False   
2          212  2014-04-04T17:07:33.339Z    10902        0  False   
3          212  2014-04-04T17:27:00.731Z    41178        0  False   
4          212  2014-04-04T17:27:26.233Z    41178        0  False   
5          212  2014-04-04T17:28:08.273Z    41216        0  False   
6          212  20